In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import gc
import glob

# 그래프 기본 테마 설정
# https://coldbrown.co.kr/2023/07/%ED%8C%8C%EC%9D%B4%EC%8D%AC-%EC%8B%A4%EC%A0%84%ED%8E%B8-08-seaborn-sns-set%EC%9D%84-%ED%86%B5%ED%95%B4-%EC%8A%A4%ED%83%80%EC%9D%BC-%EC%84%A4%EC%A0%95%ED%95%98%EA%B8%B0/
sns.set()

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False


# 복잡한 통계 처리를 위한 라이브러리
from scipy import stats

In [4]:
# data_type = "train"
# month = "07"
# category = "잔액정보"

# local
root_path = '../../data'

# colab
# root_path = '/content/drive/MyDrive/12조 파이널프로젝트/data'

#drive_folder = f'{root_path}/스탯티즈_Raw_data/'

In [7]:
df1 = pd.read_csv(f'{root_path}/KBO FA 투수 [2013-2024]_최종_전처리.csv', encoding='utf-8')

## '피장타율' 파생 컬럼
- 선수명, 플레이년도만 엑셀파일로 저장
- 수기로 피장타율 입력
- 다시 가져와서 선수명, 플레이년도로 merge

In [5]:
# template_df = df1[['선수명', '플레이년도']].drop_duplicates().sort_values(['선수명', '플레이년도'])
# template_df['피장타율_입력'] = None  # 수기로 채울 컬럼 추가

# template_df.to_excel("../../data/피장타율_입력템플릿.xlsx", index=False)

## '완투' / '완봉' 파생 컬럼

In [7]:
# template_df2 = df1[['선수명', '플레이년도']].drop_duplicates().sort_values(['선수명', '플레이년도'])
# template_df2['완투_입력'] = None  # 수기로 채울 컬럼 추가
# template_df2['완봉_입력'] = None  # 수기로 채울 컬럼 추가

# template_df2.to_excel("../../data/완투완봉_입력템플릿.xlsx", index=False)

## '사사구' 파생 컬럼
- 볼넷 + 사구

In [9]:
df1['사사구'] = df1['볼넷허용'] + df1['사구허용']

In [10]:
df1['사사구']

0      25.0
1      24.0
2      19.0
3      21.0
4      26.0
       ... 
360    59.0
361    47.0
362    18.0
363    33.0
364    16.0
Name: 사사구, Length: 365, dtype: float64

## '원클럽맨' 파생 컬럼
- 같은 선수명이 한 구단에서만 활동한 경우

In [12]:
# # 선수명 기준으로 몇 개의 구단에서 뛰었는지 count
# club_counts = df1.groupby('선수명')['구단명'].nunique()

# # 원클럽맨 여부 판단: 구단 수가 1인 경우 True
# df1['원클럽맨'] = df1['선수명'].map(lambda name: club_counts[name] == 1)

In [13]:
# df1['원클럽맨'].value_counts()

## '투구수' 파생 컬럼
- 나중에 필요시, 이닝으로 나누면 '게임당 투구수'

In [15]:
# template_df3 = df1[['선수명', '플레이년도']].drop_duplicates().sort_values(['선수명', '플레이년도'])
# template_df3['투구수_입력'] = None  # 수기로 채울 컬럼 추가

# template_df3.to_excel("../../data/투구수_입력템플릿.xlsx", index=False)

## Merge

In [17]:
# ✔ 완투완봉
df_wan = pd.read_excel('../../data/완투완봉_입력템플릿.xlsx')
df_wan = df_wan.rename(columns={
    '완투_입력': '완투',
    '완봉_입력': '완봉'
})

# ✔ 투구수
df_pitch = pd.read_excel('../../data/투구수_입력템플릿.xlsx')
df_pitch = df_pitch.rename(columns={
    '투구수_입력': '투구수'
})

# ✔ 피장타율
df_slg = pd.read_excel('../../data/피장타율_입력템플릿.xlsx')
df_slg = df_slg.rename(columns={
    '피장타율_입력': '피장타율'
})

# ✔ 원클럽맨여부
df_club = pd.read_csv('../../data/FA투수데이터_원클럽맨여부.csv')

# 필요한 컬럼만 선택
df_club = df_club[['선수명', '플레이년도', '원클럽맨여부']]

In [18]:
df1 = df1.merge(df_wan, on=['선수명', '플레이년도'], how='left')
df1 = df1.merge(df_pitch, on=['선수명', '플레이년도'], how='left')
df1 = df1.merge(df_slg, on=['선수명', '플레이년도'], how='left')
df1 = df1.merge(df_club, on=['선수명', '플레이년도'], how='left')

## 비FA 선수들의 원클럽맨여부

In [20]:
df1.loc[df1['선수명'] == '문승원', '원클럽맨여부'] = 1
df1.loc[df1['선수명'] == '박종훈', '원클럽맨여부'] = 1
df1.loc[df1['선수명'] == '박세웅', '원클럽맨여부'] = 0
df1.loc[df1['선수명'] == '김광현', '원클럽맨여부'] = 1
df1.loc[df1['선수명'] == '류현진', '원클럽맨여부'] = 1
df1.loc[df1['선수명'] == '김상수', '원클럽맨여부'] = 0
df1.loc[df1['선수명'] == '고영표', '원클럽맨여부'] = 1
df1.loc[df1['선수명'] == '구창모', '원클럽맨여부'] = 1

## 파크팩터, 국대출전선수, 구종가치

In [22]:
# ✔ 파크팩터
df_1 = pd.read_csv('../../data/파크팩터.csv', encoding='cp949')

# ✔ 국대출전선수
df_2 = pd.read_csv('../../data/국대출전선수.csv', encoding='cp949')

# ✔ 구종치치
df_3 = pd.read_csv('../../data/구종가치.csv' , encoding='utf-8')

In [23]:
df1 = df1.merge(
    df_1[['구단명', '플레이년도', '득점파크팩터', '홈런파크팩터', '장타파크팩터']],
    on=['구단명', '플레이년도'],
    how='left'
)

In [24]:
df1 = df1.merge(
    df_2[['선수명', '국대출전횟수']],
    on=['선수명'],
    how='left'
)

In [25]:
# 결측치를 0으로 채우기
df1['국대출전횟수'] = df1['국대출전횟수'].fillna(0).astype(int)

In [26]:
df1 = df1.merge(
    df_3[['선수명', '플레이년도', '구종1', '구종2', '구종1 구종가치', ]],
    on=['선수명', '플레이년도'],
    how='left'
)

In [27]:
df1

,FA년도,구단명,선수명,세부 포지션,투,구분,FA등급,플레이년도,나이,게임수,...,투구수,피장타율,원클럽맨여부,득점파크팩터,홈런파크팩터,장타파크팩터,국대출전횟수,구종1,구종2,구종1 구종가치
0,2011,롯데,강영식,RP,좌투,자격유지,A,2010,29.0,63.0,...,907,0.412,0.0,1032.0,1128.0,1093.0,0,NaN,NaN,NaN
1,2012,롯데,강영식,RP,좌투,자격유지,A,2011,30.0,64.0,...,731,0.360,0.0,1044.0,1128.0,1093.0,0,NaN,NaN,NaN
2,2013,롯데,강영식,RP,좌투,자격유지,A,2012,31.0,55.0,...,757,0.340,0.0,1033.0,1128.0,1093.0,0,NaN,NaN,NaN
3,2014,롯데,강영식,RP,좌투,자격유지,A,2013,32.0,55.0,...,675,0.298,0.0,1058.0,1128.0,1093.0,0,포심,슬라,5.0
4,2015,롯데,강영식,RP,좌투,FA 1년차,A,2014,33.0,52.0,...,813,0.393,0.0,1045.0,1102.0,1062.0,0,포심,슬라,-5.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,2019,NC,구창모,SP,좌투,다년계약,다년계약,2018,21.0,36.0,...,2383,0.483,1.0,1004.0,1093.0,992.0,1,포심,커브,2.3
361,2020,NC,구창모,SP,좌투,다년계약,다년계약,2019,22.0,23.0,...,1780,0.338,1.0,1043.0,1084.0,1018.0,1,포심,슬라,17.8
362,2021,NC,구창모,SP,좌투,다년계약,다년계약,2020,23.0,15.0,...,1378,0.278,1.0,1037.0,1091.0,1026.0,1,포심,슬라,21.9
363,2023,NC,구창모,SP,좌투,다년계약,다년계약,2022,25.0,19.0,...,1756,0.314,1.0,1013.0,1095.0,1026.0,1,포심,슬라,23.4


In [28]:
df1.to_csv('../../data/KBO FA 투수 [2013-2024]_최종_전처리+파생2.csv', index=False, encoding='utf-8-sig')